## **Recipe Recommendation System**

### **Project Goal**
To develop a recommendation system that suggests recipes to users based on their taste preferences, past ratings, and similar users’ behavior.

In [1]:
#needed libraries
import pandas as pd
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy
from sklearn.metrics.pairwise import cosine_similarity, linear_kernel
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
#load the data
recipes = pd.read_csv('recipes.csv')
reviews = pd.read_csv('reviews.csv')

In [3]:
recipes.head()

,RecipeId,Name,AuthorId,AuthorName,CookTime,PrepTime,TotalTime,DatePublished,Description,Images,...,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeYield,RecipeInstructions
0,38,Low-Fat Berry Blue Frozen Dessert,1533,Dancer,PT24H,PT45M,PT24H45M,1999-08-09T21:46:00Z,Make and share this Low-Fat Berry Blue Frozen ...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,1.3,8.0,29.8,37.1,3.6,30.2,3.2,4.0,NaN,"c(""Toss 2 cups berries with sugar."", ""Let stan..."
1,39,Biryani,1567,elly9812,PT25M,PT4H,PT4H25M,1999-08-29T13:12:00Z,Make and share this Biryani recipe from Food.com.,"c(""https://img.sndimg.com/food/image/upload/w_...",...,16.6,372.8,368.4,84.4,9.0,20.4,63.4,6.0,NaN,"c(""Soak saffron in warm milk for 5 minutes and..."
2,40,Best Lemonade,1566,Stephen Little,PT5M,PT30M,PT35M,1999-09-05T19:52:00Z,This is from one of my first Good House Keepi...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,0.0,0.0,1.8,81.5,0.4,77.2,0.3,4.0,NaN,"c(""Into a 1 quart Jar with tight fitting lid, ..."
3,41,Carina's Tofu-Vegetable Kebabs,1586,Cyclopz,PT20M,PT24H,PT24H20M,1999-09-03T14:54:00Z,This dish is best prepared a day in advance to...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,3.8,0.0,1558.6,64.2,17.3,32.1,29.3,2.0,4 kebabs,"c(""Drain the tofu, carefully squeezing out exc..."
4,42,Cabbage Soup,1538,Duckie067,PT30M,PT20M,PT50M,1999-09-19T06:19:00Z,Make and share this Cabbage Soup recipe from F...,"""https://img.sndimg.com/food/image/upload/w_55...",...,0.1,0.0,959.3,25.1,4.8,17.7,4.3,4.0,NaN,"c(""Mix everything together and bring to a boil..."


##### **Recipes Dataset Description**

The dataset contains recipes along with nutritional information, author details, and metadata. 

**RecipeId:** Unique identifier for each recipe  
**Name:** Name or title of the recipe  
**AuthorId:** Unique identifier of the recipe’s author  
**AuthorName:** Name of the recipe’s author  
**CookTime:** Cooking time in minutes  
**PrepTime:** Preparation time in minutes  
**TotalTime:** Total time required (prep + cook)  
**DatePublished:** Date the recipe was published  
**Description:** Short textual description of the recipe  
**Images:** URLs or paths to recipe images  
**RecipeCategory:** Category or type of the recipe (e.g., dessert, main course)  
**Keywords:** Keywords associated with the recipe (tags, cuisine, ingredients)  
**RecipeIngredientQuantities:** Quantity of each ingredient used  
**RecipeIngredientParts:** Names of ingredients used  
**AggregatedRating:** Average user rating for the recipe   
**ReviewCount:** Number of reviews for the recipe  
**Calories:** Energy content per serving (kcal)  
**FatContent:** Total fat content per serving (grams)  
**SaturatedFatContent:** Saturated fat content per serving (grams)  
**CholesterolContent:** Cholesterol content per serving (mg)  
**SodiumContent:** Sodium content per serving (mg)  
**CarbohydrateContent:** Carbohydrate content per serving (grams)  
**FiberContent:** Dietary fiber per serving (grams)  
**SugarContent:** Sugar content per serving (grams)  
**ProteinContent:** Protein content per serving (grams)  
**RecipeServings:** Number of servings the recipe yields  
**RecipeYield:** Yield description (e.g., “4 servings”, “1 cake”)  
**RecipeInstructions:** Step-by-step cooking instructions  


In [4]:
#dataset info
recipes.info

<bound method DataFrame.info of         RecipeId                                              Name  \
0             38                 Low-Fat Berry Blue Frozen Dessert   
1             39                                           Biryani   
2             40                                     Best Lemonade   
3             41                    Carina's Tofu-Vegetable Kebabs   
4             42                                      Cabbage Soup   
...          ...                                               ...   
522512    541379                    Meg's Fresh Ginger Gingerbread   
522513    541380  Roast Prime Rib au Poivre with Mixed Peppercorns   
522514    541381                             Kirshwasser Ice Cream   
522515    541382          Quick & Easy Asian Cucumber Salmon Rolls   
522516    541383                           Spicy Baked Scotch Eggs   

          AuthorId      AuthorName CookTime PrepTime TotalTime  \
0             1533          Dancer    PT24H    PT45M  PT24H45

In [5]:
#statistical summary
recipes.describe()

,RecipeId,AuthorId,AggregatedRating,ReviewCount,Calories,FatContent,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings
count,522517.000000,5.225170e+05,269294.000000,275028.000000,522517.000000,522517.000000,522517.000000,522517.000000,5.225170e+05,522517.000000,522517.000000,522517.000000,522517.000000,339606.000000
mean,271821.436970,4.572585e+07,4.632014,5.227784,484.438580,24.614922,9.559457,86.487003,7.672639e+02,49.089092,3.843242,21.878254,17.469510,8.606191
std,155495.878422,2.929714e+08,0.641934,20.381347,1397.116649,111.485798,46.622621,301.987009,4.203621e+03,180.822062,8.603163,142.620191,40.128837,114.319809
min,38.000000,2.700000e+01,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,1.000000
25%,137206.000000,6.947400e+04,4.500000,1.000000,174.200000,5.600000,1.500000,3.800000,1.233000e+02,12.800000,0.800000,2.500000,3.500000,4.000000
50%,271758.000000,2.389370e+05,5.000000,2.000000,317.100000,13.800000,4.700000,42.600000,3.533000e+02,28.200000,2.200000,6.400000,9.100000,6.000000
75%,406145.000000,5.658280e+05,5.000000,4.000000,529.100000,27.400000,10.800000,107.900000,7.922000e+02,51.100000,4.600000,17.900000,25.000000,8.000000
max,541383.000000,2.002886e+09,5.000000,3063.000000,612854.600000,64368.100000,26740.600000,130456.400000,1.246921e+06,108294.600000,3012.000000,90682.300000,18396.200000,32767.000000


In [6]:
reviews.head()

,ReviewId,RecipeId,AuthorId,AuthorName,Rating,Review,DateSubmitted,DateModified
0,2,992,2008,gayg msft,5,better than any you can get at a restaurant!,2000-01-25T21:44:00Z,2000-01-25T21:44:00Z
1,7,4384,1634,Bill Hilbrich,4,"I cut back on the mayo, and made up the differ...",2001-10-17T16:49:59Z,2001-10-17T16:49:59Z
2,9,4523,2046,Gay Gilmore ckpt,2,i think i did something wrong because i could ...,2000-02-25T09:00:00Z,2000-02-25T09:00:00Z
3,13,7435,1773,Malarkey Test,5,easily the best i have ever had. juicy flavor...,2000-03-13T21:15:00Z,2000-03-13T21:15:00Z
4,14,44,2085,Tony Small,5,An excellent dish.,2000-03-28T12:51:00Z,2000-03-28T12:51:00Z


##### **Reviews Dataset Description**

This dataset contains user reviews and ratings for recipes.

**ReviewId:** Unique identifier for each review  
**RecipeId:** Unique identifier of the recipe being reviewed  
**AuthorId:** Unique identifier of the user who submitted the review  
**AuthorName:** Name of the user who submitted the review  
**Rating:** Rating given by the user (numerical score)  
**Review:** Text content of the user’s review.
**DateSubmitted:** Date the review was submitted  
**DateModified:** Date the review was last modified


In [7]:
#dataset info
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1401982 entries, 0 to 1401981
Data columns (total 8 columns):
 #   Column         Non-Null Count    Dtype 
---  ------         --------------    ----- 
 0   ReviewId       1401982 non-null  int64 
 1   RecipeId       1401982 non-null  int64 
 2   AuthorId       1401982 non-null  int64 
 3   AuthorName     1401982 non-null  object
 4   Rating         1401982 non-null  int64 
 5   Review         1401768 non-null  object
 6   DateSubmitted  1401982 non-null  object
 7   DateModified   1401982 non-null  object
dtypes: int64(4), object(4)
memory usage: 85.6+ MB


In [8]:
#statistical summary 
reviews.describe()

,ReviewId,RecipeId,AuthorId,Rating
count,1.401982e+06,1.401982e+06,1.401982e+06,1.401982e+06
mean,8.179739e+05,1.526412e+05,1.558638e+08,4.407951e+00
std,5.280821e+05,1.301112e+05,5.305111e+08,1.272012e+00
min,2.000000e+00,3.800000e+01,1.533000e+03,0.000000e+00
25%,3.743862e+05,4.703875e+04,1.336800e+05,4.000000e+00
50%,7.717805e+05,1.093270e+05,3.305450e+05,5.000000e+00
75%,1.204126e+06,2.318768e+05,8.183590e+05,5.000000e+00
max,2.090347e+06,5.412980e+05,2.002902e+09,5.000000e+00


**Data Preprocessing**

In [9]:
#check for null values
recipes.isna().sum()

RecipeId                           0
Name                               0
AuthorId                           0
AuthorName                         0
CookTime                       82545
PrepTime                           0
TotalTime                          0
DatePublished                      0
Description                        5
Images                             1
RecipeCategory                   751
Keywords                       17237
RecipeIngredientQuantities         3
RecipeIngredientParts              0
AggregatedRating              253223
ReviewCount                   247489
Calories                           0
FatContent                         0
SaturatedFatContent                0
CholesterolContent                 0
SodiumContent                      0
CarbohydrateContent                0
FiberContent                       0
SugarContent                       0
ProteinContent                     0
RecipeServings                182911
RecipeYield                   348071
R

In [10]:
#drop unnecessary columns
to_drop = ['AuthorId','AuthorName','CookTime','PrepTime','TotalTime','DatePublished','Images','RecipeCategory','RecipeIngredientQuantities','AggregatedRating', 'ReviewCount', 'RecipeServings', 'RecipeYield']

recipes = recipes.drop(columns=to_drop)
recipes.head()

,RecipeId,Name,Description,Keywords,RecipeIngredientParts,Calories,FatContent,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeInstructions
0,38,Low-Fat Berry Blue Frozen Dessert,Make and share this Low-Fat Berry Blue Frozen ...,"c(""Dessert"", ""Low Protein"", ""Low Cholesterol"",...","c(""blueberries"", ""granulated sugar"", ""vanilla ...",170.9,2.5,1.3,8.0,29.8,37.1,3.6,30.2,3.2,"c(""Toss 2 cups berries with sugar."", ""Let stan..."
1,39,Biryani,Make and share this Biryani recipe from Food.com.,"c(""Chicken Thigh & Leg"", ""Chicken"", ""Poultry"",...","c(""saffron"", ""milk"", ""hot green chili peppers""...",1110.7,58.8,16.6,372.8,368.4,84.4,9.0,20.4,63.4,"c(""Soak saffron in warm milk for 5 minutes and..."
2,40,Best Lemonade,This is from one of my first Good House Keepi...,"c(""Low Protein"", ""Low Cholesterol"", ""Healthy"",...","c(""sugar"", ""lemons, rind of"", ""lemon, zest of""...",311.1,0.2,0.0,0.0,1.8,81.5,0.4,77.2,0.3,"c(""Into a 1 quart Jar with tight fitting lid, ..."
3,41,Carina's Tofu-Vegetable Kebabs,This dish is best prepared a day in advance to...,"c(""Beans"", ""Vegetable"", ""Low Cholesterol"", ""We...","c(""extra firm tofu"", ""eggplant"", ""zucchini"", ""...",536.1,24.0,3.8,0.0,1558.6,64.2,17.3,32.1,29.3,"c(""Drain the tofu, carefully squeezing out exc..."
4,42,Cabbage Soup,Make and share this Cabbage Soup recipe from F...,"c(""Low Protein"", ""Vegan"", ""Low Cholesterol"", ""...","c(""plain tomato juice"", ""cabbage"", ""onion"", ""c...",103.6,0.4,0.1,0.0,959.3,25.1,4.8,17.7,4.3,"c(""Mix everything together and bring to a boil..."


In [11]:
#fill the nulls witj an empty string
recipes['Keywords'] = recipes['Keywords'].fillna('')
recipes['Description'] = recipes['Description'].fillna('')


In [12]:
#check for null values
reviews.isna().sum()

ReviewId           0
RecipeId           0
AuthorId           0
AuthorName         0
Rating             0
Review           214
DateSubmitted      0
DateModified       0
dtype: int64

In [13]:
reviews = reviews.drop(columns='Review')
reviews

,ReviewId,RecipeId,AuthorId,AuthorName,Rating,DateSubmitted,DateModified
0,2,992,2008,gayg msft,5,2000-01-25T21:44:00Z,2000-01-25T21:44:00Z
1,7,4384,1634,Bill Hilbrich,4,2001-10-17T16:49:59Z,2001-10-17T16:49:59Z
2,9,4523,2046,Gay Gilmore ckpt,2,2000-02-25T09:00:00Z,2000-02-25T09:00:00Z
3,13,7435,1773,Malarkey Test,5,2000-03-13T21:15:00Z,2000-03-13T21:15:00Z
4,14,44,2085,Tony Small,5,2000-03-28T12:51:00Z,2000-03-28T12:51:00Z
...,...,...,...,...,...,...,...
1401977,2090339,139499,2002080368,terrylbiggs,2,2020-12-27T23:57:54Z,2020-12-27T23:57:54Z
1401978,2090340,148484,41805321,rogerberry,5,2020-12-28T00:44:42Z,2020-12-28T00:44:42Z
1401979,2090341,264191,2002901848,Reiketsukan 6.,5,2020-12-28T01:04:43Z,2020-12-28T01:04:43Z
1401980,2090345,411791,2002901938,Sue M.,5,2020-12-28T03:07:10Z,2020-12-28T03:07:10Z


**Collaborative Filtering**

In [14]:
reader = Reader(rating_scale=(1,5))
data = Dataset.load_from_df(reviews[['AuthorId','RecipeId','Rating']], reader)

trainset, testset = train_test_split(data, test_size=0.2)
algo = SVD() #matrix factorization
algo.fit(trainset)
predictions = algo.test(testset)

In [15]:
accuracy.rmse(predictions)

RMSE: 1.2283


1.228311174245103

In [16]:
accuracy.mae(predictions)

MAE:  0.7471


0.747149816184902

In [17]:

# user
user_id = '2008'

# Get all recipes the user hasn't rated
all_recipes = recipes['RecipeId'].unique()
rated_recipes = reviews[reviews['AuthorId'] == user_id]['RecipeId'].tolist()
unrated_recipes = [r for r in all_recipes if r not in rated_recipes]

# Predict ratings for all unrated recipes using CF model
predictions = [(r, algo.predict(user_id, r).est) for r in unrated_recipes]

# Sort predictions by estimated rating and take Top-N
top_n = 5
top_predictions = sorted(predictions, key=lambda x: x[1], reverse=True)[:top_n]
top_recipe_ids = [r[0] for r in top_predictions]

#  merge with recipe info 
top_recipes_info = recipes[recipes['RecipeId'].isin(top_recipe_ids)][
    ['RecipeId', 'Name', 'Calories', 'SugarContent', 'ProteinContent','FatContent']
]

print("Top-5 CF Recommended Recipes for User", user_id)
top_recipes_info


Top-5 CF Recommended Recipes for User 2008


,RecipeId,Name,Calories,SugarContent,ProteinContent,FatContent
139630,146492,Girl Scout Chocolate Mint Cookies (Copycat),39.3,2.9,0.4,2.2
179265,187415,Wait Family Pizza Dough (Bread Machine),529.6,8.6,12.5,13.3
468977,486261,Mexican Stack-Up #RSC,793.0,2.6,43.6,54.1
468982,486267,Summer Vegetable Grill Packets #RSC,232.1,7.5,5.2,14.8
477612,495202,Crunchy Valley Chicken #RSC,598.4,6.6,19.5,39.1


**Content Filtering**

In [18]:
#combine recipe text
recipes['text_features'] = (recipes['RecipeIngredientParts'] + ' ' + recipes['Keywords'] + ' ' + recipes['Description'])

In [19]:
#vectorizer
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(recipes['text_features'])

In [20]:
# Mapping RecipeId -> row index
recipe_id_to_index = {rid: idx for idx, rid in enumerate(recipes['RecipeId'])}


In [21]:
#get similar recipes
def get_similar_recipes(recipe_id, top_n=5):
    # index the recipe
    idx = recipe_id_to_index[recipe_id]
    
    # cosine similarity with all other recipes
    cosine_similarities = linear_kernel(tfidf_matrix[idx], tfidf_matrix).flatten()
    
    # Get top-N similar indices (excluding the recipe itself)
    similar_indices = cosine_similarities.argsort()[::-1][1:top_n+1]
    
    # Map back to RecipeIds
    similar_ids = recipes.iloc[similar_indices]['RecipeId']
    
    return recipes[recipes['RecipeId'].isin(similar_ids)][['RecipeId', 'Name', 'Calories', 'ProteinContent']]


In [22]:

get_similar_recipes(515167, top_n=5)


,RecipeId,Name,Calories,ProteinContent
496101,514305,Crunch and Munch Halibut Stix #SP5,193.9,34.9
496244,514451,Cheesy Shredded Hash Potato Bacon Ranch Cruste...,456.2,13.4
496575,514793,Potato Stuffed Poblano Peppers #SP5,157.4,8.1
496802,515023,Potato Artichoke Casserole #SP5,268.5,9.9
496863,515084,Maple Bacon Tater Puffs #SP5,523.3,18.4


**Hybrid recommendation**

In [23]:
def hybrid_recommendations(user_id, seed_recipe_id, top_n=5, alpha=0.6):
    # collaborative filtering
    all_recipe_ids = recipes['RecipeId'].unique()
    cf_scores={}
    for rid in all_recipe_ids:
        pred = algo.predict(user_id, rid)
        cf_scores[rid] = pred.est

    # content based filtering
    idx = recipe_id_to_index[seed_recipe_id]
    sim_scores = linear_kernel(tfidf_matrix[idx], tfidf_matrix).flatten()
    cbf_scores = {rid: sim_scores[recipe_id_to_index[rid]] for rid in all_recipe_ids}

    # cobine score
    final_scores = {}
    for rid in all_recipe_ids:
        final_scores[rid] = alpha * cf_scores[rid] + (1 - alpha) * cbf_scores[rid]

    #top N recommentations
    top_rec = sorted(final_scores.items(), key=lambda x: x[1], reverse=True)[:top_n]

    #recipe details
    top_rec_df = recipes.set_index('RecipeId').loc[[rid for rid, _ in top_rec]].reset_index()
    return top_rec_df


In [24]:
hybrid_recs = hybrid_recommendations(2008, 514451, top_n=5, alpha=0.6)
hybrid_recs[['RecipeId', 'Name', 'Calories', 'ProteinContent','SugarContent','FatContent']]


,RecipeId,Name,Calories,ProteinContent,SugarContent,FatContent
0,515084,Maple Bacon Tater Puffs #SP5,523.3,18.4,16.1,41.6
1,515167,Simply Irresistible Tropical Potato Salad #SP5,398.6,11.3,8.9,29.8
2,66229,Chicken Stew or Chicken Pot Pie Filling,241.7,25.2,0.0,15.0
3,514852,Poseidon Potato Stew #SP5,12.8,0.6,1.6,0.1
4,514451,Cheesy Shredded Hash Potato Bacon Ranch Cruste...,456.2,13.4,2.0,42.9
